# Solving Poisson's Equation on an L-shaped Domain using PINNs with DeepXDE

This notebook demonstrates how to use Physics-Informed Neural Networks (PINNs) implemented with the DeepXDE library to solve Poisson's equation on a non-trivial L-shaped domain.

**Poisson's Equation:** Δu(x, y) = f(x, y)
In this example, we will solve for f(x,y) = 1.

**L-shaped Domain:** The domain is defined as `([0,1]x[0,1]) \ ([0.5,1]x[0.5,1])`.

**Boundary Conditions:** We will apply Dirichlet boundary conditions, u(x, y) = 0, on all boundaries of the domain.

## 1. Setup: Import Libraries

In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

## 2. Geometry Definition

The L-shaped domain is defined by the following vertices: 
P1=(0,0), P2=(1,0), P3=(1,0.5), P4=(0.5,0.5), P5=(0.5,1), P6=(0,1).

In [ ]:
vertices = [[0,0], [1,0], [1,0.5], [0.5,0.5], [0.5,1], [0,1]]
geom = dde.geometry.Polygon(vertices)

# Optional: Plot geometry to verify
plt.figure(figsize=(6,6))
plt.plot(*zip(*vertices + [vertices[0]]), 'b-', label='Domain Boundary') # Close the polygon
plt.fill(*zip(*vertices), 'lightblue', alpha=0.5) # Fill the polygon
plt.xlabel('x')
plt.ylabel('y')
plt.title('L-shaped Domain Geometry')
plt.gca().set_aspect('equal', adjustable='box')
plt.legend()
plt.grid(True)
plt.show()

## 3. PDE and Boundary Condition Definition

**PDE:** Δu - 1 = 0 (Laplacian of u is equal to 1)

**Boundary Condition:** u = 0 on all boundaries.

In [ ]:
def pde(x, y):
    """Defines the Poisson equation: dy_xx + dy_yy - 1 = 0"""
    # y is the network output u(x,y)
    # x is the input tensor with x[:, 0] as x-coordinates and x[:, 1] as y-coordinates
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    dy_yy = dde.grad.hessian(y, x, i=1, j=1)
    return dy_xx + dy_yy - 1.0

def boundary_func(_, on_boundary):
    """Helper function to identify all points on the boundary."""
    return on_boundary

def func_boundary_val(x):
    """Returns the value of the solution on the boundary (0 for this problem)."""
    return np.zeros((x.shape[0], 1), dtype=dde.config.real(np))

# Create the Dirichlet Boundary Condition
bc = dde.DirichletBC(
    geom,
    func_boundary_val, 
    boundary_func
)

## 4. Problem Setup (Data)

We define the PDE problem using `dde.data.PDE`, specifying the geometry, PDE, boundary conditions, and the number of training/testing points.

In [ ]:
# Number of training points
N_domain = 2000    # Points inside the domain
N_boundary = 500   # Points on the boundary

# Number of test points for validation (monitored during training)
N_test = 500

data = dde.data.PDE(
    geom,
    pde,
    bc,
    num_domain=N_domain,
    num_boundary=N_boundary,
    num_test=N_test,
    solution=None # No analytical solution provided for testing against
)

## 5. Network and Model Creation

We define a Feed-forward Neural Network (FNN) to approximate the solution.

In [ ]:
# Network parameters
input_dim = 2  # (x, y)
output_dim = 1 # u(x,y)
num_hidden_layers = 4
neurons_per_layer = 50
activation_fn = "tanh"
kernel_initializer = "Glorot uniform"

net = dde.maps.FNN(
    [input_dim] + [neurons_per_layer] * num_hidden_layers + [output_dim],
    activation_fn,
    kernel_initializer
)

# Create the DeepXDE model
model = dde.Model(data, net)

## 6. Model Training

The model is compiled with an optimizer and learning rate, then trained for a specified number of iterations.

In [ ]:
# Compile the model
learning_rate = 1e-3
model.compile("adam", lr=learning_rate)

# Train the model
num_iterations = 20000 # Adjust as needed based on loss behavior
print(f"Starting training for {num_iterations} iterations...")
losshistory, train_state = model.train(iterations=num_iterations)
print("Training finished.")

## 7. Results and Visualization

We visualize the training progress (loss history) and the predicted solution on the L-shaped domain.

In [ ]:
# Plot loss history
dde.utils.plot_loss_history(losshistory)
plt.title('Loss History (Log Scale)')
plt.show()

# Plot the predicted solution
plot_resolution = 100
x_plot = np.linspace(0, 1, plot_resolution)
y_plot = np.linspace(0, 1, plot_resolution)
X_plot, Y_plot = np.meshgrid(x_plot, y_plot)
plot_points_flat = np.vstack((X_plot.ravel(), Y_plot.ravel())).T

# Filter points to be within the L-shaped geometry
is_inside_domain = geom.inside(plot_points_flat)
plot_points_inside = plot_points_flat[is_inside_domain]

if plot_points_inside.shape[0] > 0:
    u_pred_inside = model.predict(plot_points_inside)
    
    plt.figure(figsize=(8, 6))
    # tricontourf is good for unstructured data which plot_points_inside is
    plt.tricontourf(plot_points_inside[:, 0], plot_points_inside[:, 1], u_pred_inside.ravel(), levels=100, cmap='viridis')
    plt.colorbar(label='Predicted u(x,y)')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Predicted Solution u(x,y) on L-shaped Domain')
    plt.gca().set_aspect('equal', adjustable='box')
else:
    plt.figure(figsize=(8, 6))
    plt.text(0.5, 0.5, "No points found inside domain for plotting", ha='center', va='center')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Predicted Solution u(x,y) on L-shaped Domain')

plt.show()

## 8. Conclusion

This notebook demonstrated the setup and solution of Poisson's equation on an L-shaped domain using DeepXDE. The key steps involved:
1. Defining the complex geometry.
2. Setting up the PDE and boundary conditions.
3. Configuring the neural network and PINN model.
4. Training the model.
5. Visualizing the results (loss history and solution plot).

Further improvements or explorations could include:
- Trying different source functions `f(x,y)`.
- Experimenting with Neumann or Robin boundary conditions.
- Adjusting network architecture (layers, neurons, activation functions).
- Using adaptive learning rates or more advanced optimization schemes (e.g., L-BFGS after Adam).
- Comparing with analytical solutions if available for specific `f(x,y)` and BCs (though challenging for L-shapes).